<a href="https://www.kaggle.com/code/ahmed3okka/pg-s3-e24-modeling-submission?scriptVersionId=149614482" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import necessary libraries
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn import metrics
from xgboost import XGBClassifier
import pandas as pd

# Set XGBoost hyperparameters
n_estimators = 600
max_depth = 6

# Read the CSV files
df = pd.read_csv("/kaggle/input/playground-series-s3e24/train.csv")  # Read the first dataset
pg = pd.read_csv("/kaggle/input/smoker-status-prediction-using-biosignals/train_dataset.csv")  # Read the second dataset

# Combine the DataFrames
df = pd.concat([pg, df])  # Merge both datasets
df.head()  # Display the first few rows of the combined DataFrame

In [ ]:
# Remove the 'id' column from the DataFrame
df.drop(columns=['id'], inplace=True)

# Calculate BMI and add it as a new column in the DataFrame
df['bmi'] = df['weight(kg)'] / (df['height(cm)'] / 100) ** 2

# Store the feature names in a list, excluding the last but one column
features = list(df.columns)
features.pop(-2)  # Exclude the second-to-last column (presumably the target variable)

In [ ]:
# Split the dataset into training and validation sets (80-20 split)
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# Reset the index for both training and validation sets
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

# Extract the target variables from the training and validation sets
y_train = df_train.smoking.values
y_val = df_val.smoking.values

# Remove the 'smoking' column from both training and validation sets
del df_train['smoking']
del df_val['smoking']

In [ ]:
# Initialize an XGBoost Classifier with specified parameters
xgb_model = XGBClassifier(tree_method='hist',
                          n_estimators=600,
                          max_depth=6,
                          learning_rate=0.1,
                          colsample_bytree=0.5)

# Fit the XGBoost model on the training data
xgb_model.fit(df_train, y_train)

# Predict probabilities and actual classes on the validation set
y_pred_prob = xgb_model.predict_proba(df_val)[:, 1]  # Predict probabilities for positive class
y_pred = xgb_model.predict(df_val)  # Predict classes

# Evaluate model performance using ROC AUC and accuracy
roc = metrics.roc_auc_score(y_val, y_pred_prob)  # Compute ROC AUC score
acc = metrics.accuracy_score(y_val, y_pred)  # Compute accuracy score

# Output the ROC AUC and accuracy scores
roc, acc

In [ ]:
# Read the sample submission and test dataset for prediction
submission = pd.read_csv('/kaggle/input/playground-series-s3e24/sample_submission.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s3e24/test.csv')

# Drop the 'id' column from the test data
test_df.drop(columns=['id'], inplace=True)

# Display the first few rows of the test dataset
test_df.head()

In [ ]:
# Calculate BMI for the test dataset and add it as a new column
test_df['bmi'] = test_df['weight(kg)'] / (test_df['height(cm)'] / 100) ** 2

# Check the shape of the test dataset
test_df.shape  # Outputs the dimensions of the modified test dataset

In [ ]:
# Predict probabilities for the test dataset using the trained XGBoost model
y_test = xgb_model.predict_proba(test_df)[:, 1]

In [ ]:
# Update the 'smoking' column in the submission DataFrame with predicted values
submission['smoking'] = y_test

# Save the submission file without index to a CSV
submission.to_csv('submission.csv', index=False)